### Import essential Libraries

In [1]:
import cv2
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense

### Locate the dataset directory

In [2]:
image_directory = 'Kidney_Datasets/'

### Differentiate the datasets and traverse all images

In [3]:
normal_images = os.listdir(image_directory + 'Normal/')
stone_images = os.listdir(image_directory + 'Stone/')

In [4]:
dataset = []
label = []
INPUTSIZE = 256

### convert Images to arrays

In [5]:
for i, image_name in enumerate(normal_images):
    if(image_name.split('.')[1] == 'jpg' or 'png'): #only jpg or png images
        image = cv2.imread(image_directory + 'Normal/' + image_name)
        
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUTSIZE, INPUTSIZE))

        dataset.append(np.array(image)) #append array of images in dataset list
        label.append(0) #append '0' for all normal kidney images in label list

### convert Images to arrays create labels

In [6]:
for i, image_name in enumerate(stone_images):
    if(image_name.split('.')[1] == 'jpg' or 'png'): #only jpg or png images
        image = cv2.imread(image_directory + 'Stone/' + image_name)
        
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUTSIZE, INPUTSIZE))

        dataset.append(np.array(image)) #append array of images in dataset list
        label.append(1) #append '1' for all kidney stone images in label list

### convert list format to array format

In [7]:
dataset = np.array(dataset)
label = np.array(label)

### split dataset for training and testing

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(dataset, label, test_size=0.2, random_state=42)

# xtrain, xtest, ytrain, ytest = train_test_split(dataset, label, test_size=0.2, random_state=0)

In [9]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(960, 256, 256, 3)
(240, 256, 256, 3)
(960,)
(240,)


### **normalize** the data in xtrain and xtest, where normalize is a function that likely comes from sklearn.preprocessing or a similar library.

Why Normalize?
 * Avoid Bias in Features: If one feature has much larger values than others, it could dominate the model during training. Normalizing ensures that all features contribute equally.

 * Improved Performance: Many machine learning algorithms (such as KNN, SVM, etc.) perform better when the data is normalized because these algorithms rely on distance-based measures or optimization techniques that benefit from standardized data ranges.
 
 
 

In [10]:
xtrain = normalize(xtrain, axis=1)
xtest = normalize(xtest, axis=1)

### Build CNN Model

In [11]:
# #Basic CNN Model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (INPUTSIZE, INPUTSIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(256)) #corresponding to INPUTSIZE
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

############################RESNET50#################################
# from keras.applications import ResNet50
# from keras.models import Sequential
# from keras.layers import Dense, GlobalAveragePooling2D
# from keras.optimizers import Adam

# # Load ResNet50 model (pre-trained on ImageNet)
# resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(INPUTSIZE, INPUTSIZE, 3))

# # Freeze the layers in the pre-trained model
# for layer in resnet_model.layers:
#     layer.trainable = False

# # Add custom top layers for our specific task
# model = Sequential()
# model.add(resnet_model)
# model.add(GlobalAveragePooling2D())
# model.add(Dense(256, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss= 'binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [13]:
model.fit(xtrain, ytrain, 
          batch_size=16, 
          verbose=1, 
          epochs=3, 
          validation_data=(xtest, ytest), 
          shuffle=False)

Epoch 1/3
60/60 [==============================] - 110s 2s/step - loss: 0.0786 - accuracy: 0.9719 - val_loss: 3.5804e-04 - val_accuracy: 1.0000
Epoch 2/3
60/60 [==============================] - 107s 2s/step - loss: 1.2110e-04 - accuracy: 1.0000 - val_loss: 2.8771e-07 - val_accuracy: 1.0000
Epoch 3/3
60/60 [==============================] - 103s 2s/step - loss: 6.8092e-06 - accuracy: 1.0000 - val_loss: 3.5767e-08 - val_accuracy: 1.0000


In [14]:
model.save('kidney_stone_CNN2.h5')

In [15]:
test_loss, test_acc = model.evaluate(xtest, ytest)
print(f"This Model error is: {round((test_loss)*100, 2)}% error")
print(f"This Model scores: {round((test_acc)*100, 2)}% accuracy")

8/8 [==============================] - 8s 836ms/step - loss: 3.5767e-08 - accuracy: 1.0000
This Model error is: 0.0% error
This Model scores: 100.0% accuracy


# Graph Plotting

In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(xtrain, ytrain, 
          batch_size=16, 
          verbose=1, 
          epochs=3, 
          validation_data=(xtest, ytest), 
          shuffle=False)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')

fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')

fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()